In [ ]:
# T bounds 324-524 K
# P bounds 0.1-36 atm
# Steady-state operation with ideal mixing, ideal phase equilibrium, and ideal gas behaviour.
import numpy as np
# stream and process unit tracker class
class Streams():
    _instances = []
    def __init__(self,name):
        self.__class__._instances.append(self)
        self.name = name
    @classmethod
    def get_all(cls):
        return cls._instances

    @classmethod
    def clear_registry(cls):
        cls._instances = []


class Stream(Streams):
    def __init__(self, name, T, P, F,z=[0,0,0,0]):
        # 1 Ethylene Oxide
        # 2 Water
        # 3 Ethylene Glycol
        # 4 Diethylene Glycol
        super().__init__(name)
        self.T = T
        self.P = P
        self.F = F
        self.z = np.array(z)
        self.x = z
        self.y = np.zeros(4)
    def __str__(self):
        return f"Stream {self.name}: T={self.T} K, P={self.P} atm, F={self.F} kmol/h, z={self.z}, x={self.x}, y={self.y}"

    def saturation_pressure(self):
        sat_data = np.array([
            [10.884,3152,7.667],
            [11.680,3828,-45.412],
            [11.963,4764,-72.275],
            [11.256,4655,103.551],
            ])
        psats= np.exp(sat_data[:,0] - sat_data[:,1]/(sat_data[:,2]+self.T))
        return psats
        
    def liquid_vapor_molar_fractions(self):
        lmdp = np.array([
            [1.655, 0.126300,-0.0002286],
            [75.22, -0.065050, 0],
            [23.66, -0.018040, 0],
            [13.59, -0.009986, 0],
            ])
        
        liquid_molar_densities = lmdp[:,0] + lmdp[:,1]*self.T + lmdp[:,2]*self.T**2

    def H(self):
        hl_data = np.array([
            [-143.8, 0.2037],
            [-320, 0.1068],
            [-519, 0.1924],
            [-723, 0.3043],
            ])

        hv_data = np.array([
            [-74.6, 0.0697],
            [-252.3, 0.0348],
            [-424.3, 0.1030],
            [-603.4, 0.1823],
            ])
        
        hls = hl_data[:,0] + hl_data[:,1]*self.T
        hvs = hv_data[:,0] + hv_data[:,1]*self.T
        
        hl = sum(hls*self.x)
        hv = sum(hvs*self.y)
        return self.F*hl

class ProcessUnits():
    _instances = []
    def __init__(self, name):
        self.__class__._instances.append(self)
        self.name = name

    @classmethod
    def get_all(cls):
        return cls._instances.copy()

    @classmethod
    def clear_registry(cls):
        cls._instances = []
        
class Heater(ProcessUnits):
    def __init__(self, name, inlet_stream, Tout):
        super().__init__(name)
        if inlet_stream.T > Tout:
            raise ValueError("Heater inlet temperature must be less than outlet temperature.")
        self.Tout = Tout
        self.outlet= Stream(name+"_outlet", Tout, inlet_stream.P, inlet_stream.F, inlet_stream.z)
        self.Q = self.outlet.H() - inlet_stream.H()

class Cooler(ProcessUnits):
    def __init__(self, name, inlet_stream, Tout):
        super().__init__(name)
        if inlet_stream.T < Tout:
            raise ValueError("Cooler inlet temperature must be greater than outlet temperature.")
        self.Tout = Tout
        self.outlet= Stream(name+"_outlet", Tout, inlet_stream.P, inlet_stream.F, inlet_stream.z)
        self.Q = inlet_stream.H() - self.outlet.H()

class Mixer(ProcessUnits):
    def __init__(self, name, inlet_stream1, inlet_stream2, inlet_stream3=None, inlet_stream4=None):
        super().__init__(name)
        self.inlet_stream1 = inlet_stream1
        self.inlet_stream2 = inlet_stream2
        self.inlet_stream3 = inlet_stream3
        self.inlet_stream4 = inlet_stream4
        P = min(inlet_stream1.P, inlet_stream2.P, (inlet_stream3.P if inlet_stream3 else float('inf')), (inlet_stream4.P if inlet_stream4 else float('inf')))
        F = inlet_stream1.F + inlet_stream2.F + (inlet_stream3.F if inlet_stream3 else 0) + (inlet_stream4.F if inlet_stream4 else 0)
        T = (inlet_stream1.F*inlet_stream1.T + inlet_stream2.F*inlet_stream2.T + (inlet_stream3.F*inlet_stream3.T if inlet_stream3 else 0) + (inlet_stream4.F*inlet_stream4.T if inlet_stream4 else 0)) / F
        z = (inlet_stream1.F*inlet_stream1.z + inlet_stream2.F*inlet_stream2.z + (inlet_stream3.F*inlet_stream3.z if inlet_stream3 else 0) + (inlet_stream4.F*inlet_stream4.z if inlet_stream4 else 0)) / F
        self.outlet = Stream(name+"_outlet", T, P, F, z)

class CSTReactor(ProcessUnits):
    def __init__(self, name, inlet_stream,volume=0):
        super().__init__(name)
        self.inlet_stream = inlet_stream
        self.volume = volume
        c10,c20,c30,c40 = inlet_stream.x/self.volume
        rate_main = 13706.91*np.exp(-8220/inlet_stream.T)*inlet_stream.z[0]*inlet_stream.z[1]
        rate_side = 96341.59*np.exp(-8700/inlet_stream.T)*inlet_stream.z[0]*inlet_stream.z[2]

In [181]:
# T Kelvin, P atm, n kmol/h, z1 mol fraction of Ethylene oxide, z2 mol fraction of water
Feed1 = Stream('Feed1', 298,2.4,26.32,z=[0,1,0,0])
Feed2 = Stream('Feed2', 298,2.4,27.62,z=[1,0,0,0])
D1 = Stream('D1', 360.5,2.4,528.018,z=[0.0088,0.9911,0.0001,0])
Mixer1 = Mixer('Mixer1', Feed1, Feed2, D1)
print(Mixer1.outlet.T)
Heater1 = Heater('Heater1', Feed1, 355)
print(Heater1.outlet.H())
print(Heater1.Q)


354.70705617931185
-7424.50352
160.2256320000006
